# Simple Arrays

In [1]:
import metadsl
import typing

In [2]:
class Array(metadsl.Instance):
    @metadsl.call(lambda self, other: Array)
    def __add__(self, other: "Array") -> "Array":
        ...
    
    @metadsl.call(lambda self, idx: Array)
    def __getitem__(self, idx: int) -> "Array":
        ...

array_replacements = metadsl.Replacements()


@array_replacements.register_pure(Array, Array, None)
def _distribute_add(a: Array, b: Array, c: int):
    return (
        (a + b)[c],
        a[c] + b[c]
    )

In [3]:
array_string_replacements = metadsl.Replacements()


all_replacements = metadsl.ReplacementsSequence(array_replacements, array_string_replacements)

@metadsl.call(lambda x: Array)
def array(x: str) -> Array:
    ...


@metadsl.call(lambda x: all_replacements)
def as_string(x: Array) -> str:
    ...

    
@array_string_replacements.register_pure(None)
def _string(x: str):
    return as_string(array(x)), x

@array_string_replacements.register(None, None)
def _add(x: str, y: str):
    return array(x) + array(y), lambda: array(f"({x} + {y})")


@array_string_replacements.register(None, None)
def _getitem(x: str, y: int):
    return array(x)[y], lambda: array(f"{x}[{y}]")

In [5]:
a = array("a")
b = array("B")

res = (a + b)[10]
print(as_string(res))

(a[10] + B[10])
